In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS

# 1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object. 

In [ ]:
URL = 'https://realpython.github.io/fake-jobs/'
response = requests.get(URL)
response

In [ ]:
soup = BS(response.text)
soup

In [ ]:
html = soup.prettify()
print(html)

### a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.

In [ ]:
soup.find('h2')

In [ ]:
soup.find('h2').text

### b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.

In [ ]:
jobs = soup.findAll('h2')

In [ ]:
job_titles = []
for job in jobs:
    job_titles.append(job.text)

In [ ]:
job_titles

### c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.

In [ ]:
locations = soup.findAll('p', {'class': 'location'})
# Answer found at: https://stackoverflow.com/questions/5041008/how-to-find-elements-by-class

In [ ]:
loc_list = []
for loc in locations:
    #print(loc.text.strip('\n').strip())
    loc_list.append(loc.text.strip('\n').strip())

In [ ]:
loc_list

In [ ]:
times = soup.findAll('time')
post_dates = []

In [ ]:
for time in times:
    #print(time.text)
    post_dates.append(time.text)

In [ ]:
post_dates

In [ ]:
companies = soup.findAll('h3', {'class': 'subtitle'})
comp_list = []

In [ ]:
for comp in companies:
    #print(comp.text)
    comp_list.append(comp.text)

In [ ]:
comp_list

### d. Take the lists that you have created and combine them into a pandas DataFrame.

In [ ]:
jobs_df = pd.DataFrame(job_titles)
jobs_df.columns = ['Title']
jobs_df

In [ ]:
jobs_df['Location'] = loc_list
jobs_df['Company'] = comp_list
jobs_df['Post_Date'] = post_dates
jobs_df

# 2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.
###  a. First, use the BeautifulSoup find_all method to extract the urls.

In [ ]:
apply = soup.findAll('a')
apply

In [ ]:
apply[1].text

In [ ]:
apply[0].get('href')

In [ ]:
apply_link = []

for link in apply:
    if link.text == 'Apply':
        #print(link.get('href'))
        apply_link.append(link.get('href').strip())

In [ ]:
apply_link

In [ ]:
jobs_df['Apply_Link'] = apply_link
jobs_df

### b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [ ]:
import re

In [ ]:
i = 0
concat_links = []

for row, data in jobs_df.iterrows():
    #print(data['Title'],'\n')
    title = data['Title'].lower()
    title = re.subn('[-,/()]', ' ', title)[0]
    # subn round at: https://www.geeksforgeeks.org/python-replace-multiple-characters-at-once/
    title = title.strip().replace('  ', ' ').replace(' ', '-')
    #print(title)
    
    new_link = f'https://realpython.github.io/fake-jobs/jobs/{title}-{row}.html'
    concat_links.append(new_link)
    
concat_links

In [ ]:
jobs_df['Apply_Link'] = concat_links
jobs_df

# 3. Finally, we want to get the job description text for each job.
### a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.

In [ ]:
URL = 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'
response = requests.get(URL)
response

In [ ]:
soup = BS(response.text)
soup

In [ ]:
strings = soup.findAll('p')
strings

In [ ]:
strings[1].text

### b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".

In [ ]:
def get_descr(URL):
    response = requests.get(URL)
    soup = BS(response.text)
    
    descr = soup.findAll('p')[1].text
    return descr

In [ ]:
get_descr('https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html')

### c. Use the .apply method on the url column you created above to retrieve the description text for all of the jobs.

In [ ]:
descriptions = jobs_df['Apply_Link'].apply(get_descr)

In [ ]:
jobs_df['Description'] = descriptions
jobs_df